# Parameter grids, parallel and resumed execution

In [1]:
# Imports
from mltraq import Run, create_experiment

In [2]:
def f1(run: Run):
    """
    Store inputs as fields and compute AB
    """
    run.fields.A = run.params.A
    run.fields.B = run.params.B
    run.fields.C = run.config.C
    run.fields.AB = run.fields.A + run.fields.B

In [3]:
def f2(run: Run):
    """
    Compute ABC
    """
    run.fields.ABC = run.fields.AB + run.fields.C

In [4]:
def f3(run: Run):
    """
    Compute ABCD
    """
    run.fields.ABCD = run.fields.ABC + run.config.D

In [5]:
# Create experiment
experiment = create_experiment("example")

In [6]:
(
    experiment.add_runs(A=[1, 2], B=[3, 4])  # Parameters grid
    .execute([f1, f2], config={"C": 5})  # Execute steps
    .persist()
)  # Persistence to database

 25%|##5       | 1/4 [00:00<00:01,  1.54it/s]

In [7]:
(
    experiment.reload()  # Reload experiment from database
    .execute(f3, config={"D": 6})  # Continue execution
    .persist(if_exists="replace")  # Persist to database
    .db.query("SELECT A, B, C, AB, ABC, ABCD FROM experiment_example")
)  # SQL query

 25%|##5       | 1/4 [00:00<00:00, 97.78it/s]

,A,B,C,AB,ABC,ABCD
0,2,3,5,5,10,16
1,2,4,5,6,11,17
2,1,4,5,5,10,16
3,1,3,5,4,9,15
